In [1]:
# import modules
from datasets import load_dataset
from transformers import LlamaForCausalLM, LlamaTokenizer
import argparse
import socket
from dataset import get_dataset, build_dataset
import torch
import os
import json

/home/laal_intern003/anaconda/envs/legal-master/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
path = './dataset'
if not os.path.exists(path):
    os.mkdir(path)

dataset = get_dataset('case_hold')

Found cached dataset lex_glue (/home/laal_intern003/.cache/huggingface/datasets/lex_glue/case_hold/1.0.0/8a66420941bf6e77a7ddd4da4d3bfb7ba88ef48c1d55302a568ac650a095ca3a)
100%|██████████| 3/3 [00:00<00:00, 375.51it/s]


In [10]:
dataset['train'].features.keys()



dict_keys(['context', 'endings', 'label'])

In [13]:
type(dataset['train']['context'])

data = {}
for split in ['train', 'test', 'validation']:
    keys = dataset[f'{split}'].features.keys()
    data[f'{split}'] = {}

    for key in keys:
        data[f'{split}'][f'{key}'] = dataset[f'{split}'][key]

list